In [2]:
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D 
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import efficientnet.keras as enet
import keras_applications as app
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model

#%matplotlib inline
import math 
import datetime
import time
CUDA_VISIBLE_DEVICES = 0


Using TensorFlow backend.
/home/hades/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hades/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hades/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hades/.local/lib/python3.6/site-packages/

In [3]:

#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = "bottleneck_fc_model_50_50.h5"
# loading up our datasets
train_data_dir = '/media/hades/DISK 0 - Drive 3/Fish/Test7/Train' 
validation_data_dir = '/media/hades/DISK 0 - Drive 3/Fish/Test7/Validate' 
test_data_dir = '/media/hades/DISK 0 - Drive 3/Fish/Test7/Test'
 
# number of epochs to train top model 
epochs = 50 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 50 


In [4]:
import efficientnet.keras as enet

#Loading vgc16 model
print("HAS")
vgg16 = app.mobilenet_v2.MobileNetV2(utils = keras.utils, models=keras.models, layers = keras.layers, backend = keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
#vgg16 = app.vgg16.VGG16(utils = keras.utils, models=keras.models, layers = keras.layers, backend = keras.backend, include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')
#vgg16=app.resnet.ResNet50(utils = keras.utils, models=keras.models, layers = keras.layers, include_top=False,input_shape=(224,224,3),weights='imagenet',pooling='avg', backend = keras.backend)
#vgg16 = app.MobileNet(include_top=False, input_shape=(224,224,3), pooling='avg', weights='imagenet')

datagen = ImageDataGenerator(rescale=1. / 255,
                             rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
#needed to create the bottleneck .npy files

HAS








In [5]:
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
print("ASD1")
generator = datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) 
vgg16.LOAD_TRUNCATED_IMAGES = True

np.save('bottleneck_features_train_50_50.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

ASD1


FileNotFoundError: [Errno 2] No such file or directory: '/media/hades/DISK 0 - Drive 3/Fish/Test7/Train'

In [ ]:
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
    test_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_validation_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size)) 
 
bottleneck_features_validation = vgg16.predict_generator(generator, predict_size_validation) 
 
np.save('bottleneck_features_validation_50_50.npy', bottleneck_features_validation)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
#training data
generator_top = datagen.flow_from_directory( 
   train_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train_50_50.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)


In [ ]:
#training data
generator_top = datagen.flow_from_directory( 
   test_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_validation_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
validation_data = np.load('bottleneck_features_validation_50_50.npy') 
 
# get the class labels for the training data, in the original order 
validation_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

#validation_data = validation_data.reshape((-1, 224, 224, 3))

In [ ]:
# Swish defination
from keras.backend import sigmoid

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

In [ ]:
train_data.shape

In [ ]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D 
from keras.layers import Convolution2D,MaxPooling2D,Dropout,Dense,Flatten,BatchNormalization,Conv2D
start = datetime.datetime.now()
#print(train_data[2].shape[1])
# model = Sequential() 
# #model.add(Flatten(input_shape=train_data.shape[1:])) 
# model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5)) 
# model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3)) 
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
#    metrics=['acc'])

# model.add(Dense(200, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5)) 
# model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3))
# model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.3))
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
# #    metrics=['acc'])


model = Sequential() 
model.add(BatchNormalization()) 
model.add(Dropout(0.3)) 
model.add(Dense(128))
model.add(BatchNormalization()) 
model.add(Activation(swish_act))
model.add(Dropout(0.1)) 
model.add(Dense(64)) 
model.add(BatchNormalization()) 
model.add(Activation(swish_act))
model.add(Dense(num_classes, activation="softmax"))

# model.compile(loss='categorical_crossentropy',
#    optimizer=optimizers.Adam(lr=1e-4),
#    metrics=['acc'])



# model = Sequential()

# # building 2 fully connected layer 
# #x = model.output

# x = BatchNormalization()(x)
# x = Dropout(0.7)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)
# x = Dropout(0.5)(x)

# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)

# output layer
#predictions = Dense(10, activation="softmax")(x)

#model_final = Model(inputs = model.input, outputs = predictions)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(0.0001),
              metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, verbose=1,)
#mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')

history = model.fit(train_data, train_labels, 
   epochs=1000,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels), callbacks=[reduce_lr])

model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print('[INFO] Loss: {}'.format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
model.summary()

In [ ]:
#Graphing our training and validation
#print(history.history)
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
hasil=os.listdir("/media/hades/DISK 1 - Drive 2/Fish/Test/Test/")

consumable=pd.read_csv("Commercial.txt",delimiter=r"\t+")
consumable2=pd.read_csv("Game_Fishes.txt",delimiter=r"\t+")

unconsumable=pd.read_csv("Dangerous.txt",delimiter=r"\t+")


unconsumable_final=[]
consumable_final=[]
consumable2_final=[]

for x in consumable['Species']:
    x=x.replace(' ','_')
    consumable_final.append(x)
for x in unconsumable['Species']:
    x=x.replace(' ','_')
    unconsumable_final.append(x)
for x in consumable2['Species']:
    x=x.replace(' ','_')
    consumable2_final.append(x)

In [ ]:
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2],reverse=True)  
    return tup  

In [ ]:
def read_image(file_path):
   #print('[INFO] loading and preprocessing image…') 
   image = load_img(file_path, target_size=(224, 224)) 
   image = img_to_array(image) 
   image = np.expand_dims(image, axis=0)
   image /= 255. 
   return image
def test_single_image(path):
  animals = os.listdir('/media/hades/DISK 1 - Drive 2/Fish/Test/Train')
  animals=sorted(animals)
  images = read_image(path)
  time.sleep(.5)
  bt_prediction = vgg16.predict(images) 
  preds = model.predict_proba(bt_prediction)
  #print("preds:",preds)
  finalpercent=Sort_Tuple(list(zip(range(0,667), animals , preds[0])))
  #print(finalpercent)
  #for idx, animal, x in zip(range(0,10), animals , preds[0]):
   #print('ID: {}, Label: {} {}%'.format(idx, animal, round(x*100,2) ))
    
  #print('Final Decision:')
  time.sleep(.5)
  for x in range(3):
   #print('.'*(x+1))
   time.sleep(.2)
  class_predicted = model.predict_classes(bt_prediction)
  class_dictionary = generator_top.class_indices 
  inv_map = {v: k for k, v in class_dictionary.items()} 
  #print("PRED:",class_dictionary.items())
  finalpred='ID: {}, Label: {}'.format(class_predicted[0],  inv_map[class_predicted[0]])
  finalpred2=[]
  for x,y in enumerate(finalpercent[:5]):
        finalpred2.append('ID: {}, Label: {} {}%'.format(y[0],y[1],y[2]))
  #print(finalpred2)
  #print('ID: {}, Label: {}'.format(class_predicted[0],  inv_map[class_predicted[0]])) 
  return finalpred,finalpred2,finalpercent

path = '/media/hades/DISK 1 - Drive 2/Fish/Test/Test/Naso_lituratus/Naso_lituratus_0063.jpg'


In [ ]:
def get_data(species):
    nama_species=species[0][1].replace("_",' ')
    print("namaSP:",species[0][1])
    pd.set_option('display.max_rows', unconsumable.shape[0]+1)
    print(unconsumable)
    if species[0][1] in consumable_final:
        row=consumable.loc[consumable['Species'] == nama_species]
        row_list=[]
        
        print("Tes1:",row['Species'])
        row_list.append("CONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['Use'].values[0])
    elif species[0][1] in consumable2_final:
        row=consumable2.loc[consumable2['Species'] == nama_species]
        row_list=[]
        
        print("Tes2:",row['Species'])
        row_list.append("CONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['No'].values[0])
    else:
        print("ASDDDDDDDDDDDDDDDD",nama_species)
        row=unconsumable.loc[unconsumable['Species'] == nama_species]
        row_list=[]
        
        print("Tes3:",row['Species'])
        row_list.append("UNCONSUMABLE")
        row_list.append("Species Name: %s" %row['Species'].values[0])
        row_list.append("Order: %s" %row['Order'].values[0])
        row_list.append("Family: %s" %row['Family'].values[0])
        row_list.append("Foreign Name: %s" %row['FishBase name'].values[0])
        row_list.append("Local Name: %s" %row['Name'].values[0])
        row_list.append("Description: %s" %row['Danger'].values[0])
        
    return row_list

In [ ]:
import matplotlib.image as mpimg
%matplotlib tk

img = mpimg.imread(path)
plt.imshow(img)
a,b,c=test_single_image(path)
print(get_data(c))
textstr = '\n'.join((
    r'Classification Result',
        r'',
    r'',
    r'Final Result:',
        r'',
        r'%s'%a,
        r'',
        r'DEBUG: ',
        r'',
        r'%s'%b[0],
        r'%s'%b[1],
        r'%s'%b[2],
        r'%s'%b[3],
        r'%s'%b[4]
    ))
d=get_data(c)
print(get_data(c))
#print(get_data(c))
textstr2 = '\n'.join((
        r'%s'%d[0],
        r'Details',
        r'',
        r'%s'%d[1],
        r'%s'%d[2],
        r'%s'%d[3],
        r'%s'%d[4],
        r'%s'%d[5],
        r'%s'%d[6],
        
    ))
plt.xlabel(' ')
plt.ylabel(' ')






props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.subplots_adjust(left=0.35)

plt.text(0.05, 0.95, textstr, transform=plt.gcf().transFigure, fontsize=10,
        verticalalignment='top', bbox=props)

plt.text(0.05, 0.42, textstr2, transform=plt.gcf().transFigure, fontsize=10,
        verticalalignment='top', bbox=props)

plt.show()


